**Rearrange data**

In [1]:
import numpy as np
import scipy as sp
import seaborn as sns
import pandas as pd
import scipy
import sklearn
import os
import mne
import networkx as nx
# import pycircstat as ps
from matplotlib import pyplot as plt
from scipy.io import loadmat


In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
folder            = '/Users/ren/Projects-NeuroCode/MyExperiment/Aging-SeqMemTask' #'/Users/renxiangjuan/Nextcloud/MATLAB/MyExperiment/Aging-SeqMemTask'
data_behv         = folder + '/AgingReplay-OnlineData/' 
data_behv_inHouse = folder + '/AgingReplay-InHouseData/' 
statSaveDir       = folder + '/AgingReplay-StatAnal/' 
modelSaveDir      = folder + '/ModelFitting_Results/'

# Experiment 1: memory gradation across content, position and reconstruction report in both younger (18-30 ys) and older (>75 ys) adults

<div class="alert alert-success">
<b>(Columns) variables</b>:
    
{0: 'groupNo', 1: 'subjNo', 2: 'trialNo', 3: 'measNo', 4: 'accuracy', 5: 'fullyCor'}

groupNo: younger or older
    
measNo: content, position, and reconstruction
</div>


## Figure 2A and 2B-left panel: mixture comparison –– within age group comparison with trial numbers and response types as regressor & between age group comparsion

coding each response per slot as 1 (correct) or 0 (incorrect)

The data wass originally saved in seqMemTask_v1_anal_summary.m

In [4]:
respMat = loadmat(data_behv + 'resp_glmm_group_exp1.mat') # 'resp_glmm_group'
resp_glmm_group = respMat['resp_glmm_group']
resp_glmm_pd    = pd.DataFrame(resp_glmm_group)
resp_glmm_pd

,0,1,2,3,4,5
0,0,0,1,1,0,1
1,0,0,1,2,0,1
2,0,0,1,3,0,1
3,0,0,1,4,0,1
4,0,0,1,5,0,1
...,...,...,...,...,...,...
63355,1,26,64,1,2,0
63356,1,26,64,2,2,0
63357,1,26,64,3,2,0
63358,1,26,64,4,2,0


In [5]:
column_name = {0: 'groupNo', 1: 'subjNo', 2: 'trialNo', 3: 'transNo', 4: 'measNo', 5: 'resp'} 
resp_glmm_pd = resp_glmm_pd.rename(columns = column_name) 
resp_glmm_pd.groupNo = resp_glmm_pd.groupNo.astype('int64')
resp_glmm_pd.subjNo  = resp_glmm_pd.subjNo.astype('int64')
resp_glmm_pd.trialNo = resp_glmm_pd.trialNo.astype('int64')
resp_glmm_pd.transNo = resp_glmm_pd.transNo.astype('int64')
resp_glmm_pd.measNo  = resp_glmm_pd.measNo.astype('int64')
resp_glmm_pd.resp    = resp_glmm_pd.resp.astype('int64')
resp_glmm_pd

,groupNo,subjNo,trialNo,transNo,measNo,resp
0,0,0,1,1,0,1
1,0,0,1,2,0,1
2,0,0,1,3,0,1
3,0,0,1,4,0,1
4,0,0,1,5,0,1
...,...,...,...,...,...,...
63355,1,26,64,1,2,0
63356,1,26,64,2,2,0
63357,1,26,64,3,2,0
63358,1,26,64,4,2,0


In [6]:
nGroup = 2 # younger and older groups
nMeas  = 3 # three measures: content, position and reconstruction
groupList = ['0younger', '1older']
measList  = ['0content', '1position', '2both']
for iM in range(0, nMeas):
    resp_glmm_pd.loc[(resp_glmm_pd["measNo"] == iM), "measNo"] = measList[iM]
for iGrp in range(0, nGroup):
    resp_glmm_pd.loc[(resp_glmm_pd["groupNo"] == iGrp), "groupNo"] = groupList[iGrp]  
resp_glmm_pd


,groupNo,subjNo,trialNo,transNo,measNo,resp
0,0younger,0,1,1,0content,1
1,0younger,0,1,2,0content,1
2,0younger,0,1,3,0content,1
3,0younger,0,1,4,0content,1
4,0younger,0,1,5,0content,1
...,...,...,...,...,...,...
63355,1older,26,64,1,2both,0
63356,1older,26,64,2,2both,0
63357,1older,26,64,3,2both,0
63358,1older,26,64,4,2both,0


In [7]:
# normalize the trial numbers for YAs and OAs separately and then concatenate 
resp_glmm_pd_YA = resp_glmm_pd[resp_glmm_pd['groupNo'] == '0younger'].copy()
resp_glmm_pd_YA['trialNo'] = np.log(resp_glmm_pd_YA['trialNo'])

resp_glmm_pd_OA = resp_glmm_pd[resp_glmm_pd['groupNo'] == '1older'].copy()
resp_glmm_pd_OA['trialNo'] = np.log(resp_glmm_pd_OA['trialNo'])

In [8]:
resp_glmm_pd = pd.concat([resp_glmm_pd_YA, resp_glmm_pd_OA], ignore_index=True)
resp_glmm_pd

,groupNo,subjNo,trialNo,transNo,measNo,resp
0,0younger,0,0.000000,1,0content,1
1,0younger,0,0.000000,2,0content,1
2,0younger,0,0.000000,3,0content,1
3,0younger,0,0.000000,4,0content,1
4,0younger,0,0.000000,5,0content,1
...,...,...,...,...,...,...
63355,1older,26,4.158883,1,2both,0
63356,1older,26,4.158883,2,2both,0
63357,1older,26,4.158883,3,2both,0
63358,1older,26,4.158883,4,2both,0


In [9]:
path_or_buf = data_behv + 'resp_glmm_group_exp1.csv' # save in the folder of online data
resp_glmm_pd.to_csv(path_or_buf = path_or_buf)

## Figure 2B-right panel: Comparison between the accuracy changes (acc in older minus mean of younger)

The data was originally saved in seqMemTask_v1_anal_summary.m

In [4]:
acc_change_Mat = loadmat(data_behv + 'acc_change_lmm_subj_exp1.mat') # 'acc_change_lmm_subj'
acc_lmm_change = acc_change_Mat['acc_change_lmm_subj']
acc_lmm_pd    = pd.DataFrame(acc_lmm_change) 
acc_lmm_pd

,0,1,2,3,4,5
0,1.0,0.0,1.0,0.0,0.079772,0.0
1,1.0,0.0,2.0,0.0,-0.352137,0.0
2,1.0,0.0,3.0,0.0,-0.784046,0.0
3,1.0,0.0,4.0,0.0,0.079772,0.0
4,1.0,0.0,5.0,0.0,-0.136183,0.0
...,...,...,...,...,...,...
5179,1.0,26.0,60.0,2.0,-1.000000,0.0
5180,1.0,26.0,61.0,2.0,-0.209459,0.0
5181,1.0,26.0,62.0,2.0,-0.736486,0.0
5182,1.0,26.0,63.0,2.0,-0.736486,0.0


In [5]:
column_name = {0: 'groupNo', 1: 'subjNo', 2: 'trialNo', 3: 'measNo', 4: 'acc_change', 5: 'fullyCor'} 
acc_lmm_pd = acc_lmm_pd.rename(columns = column_name) 
acc_lmm_pd.groupNo = acc_lmm_pd.groupNo.astype('int64')
acc_lmm_pd.subjNo  = acc_lmm_pd.subjNo.astype('int64')
acc_lmm_pd.trialNo = acc_lmm_pd.trialNo.astype('int64')
acc_lmm_pd.measNo  = acc_lmm_pd.measNo.astype('int64')
acc_lmm_pd.fullyCor = acc_lmm_pd.fullyCor.astype('int64')
acc_lmm_pd

,groupNo,subjNo,trialNo,measNo,acc_change,fullyCor
0,1,0,1,0,0.079772,0
1,1,0,2,0,-0.352137,0
2,1,0,3,0,-0.784046,0
3,1,0,4,0,0.079772,0
4,1,0,5,0,-0.136183,0
...,...,...,...,...,...,...
5179,1,26,60,2,-1.000000,0
5180,1,26,61,2,-0.209459,0
5181,1,26,62,2,-0.736486,0
5182,1,26,63,2,-0.736486,0


In [6]:
nGroup = 1 # older relative to younger
nMeas  = 3 # three measures: content, position and reconstruction
groupList = ['older-younger']
measList  = ['0content', '1position', '2both']
for iM in range(0, nMeas):
    acc_lmm_pd.loc[(acc_lmm_pd["measNo"] == iM), "measNo"] = measList[iM]
for iGrp in range(0, nGroup):
    acc_lmm_pd.loc[(acc_lmm_pd["groupNo"] == iGrp), "groupNo"] = groupList[iGrp]  
acc_lmm_pd

,groupNo,subjNo,trialNo,measNo,acc_change,fullyCor
0,1,0,1,0content,0.079772,0
1,1,0,2,0content,-0.352137,0
2,1,0,3,0content,-0.784046,0
3,1,0,4,0content,0.079772,0
4,1,0,5,0content,-0.136183,0
...,...,...,...,...,...,...
5179,1,26,60,2both,-1.000000,0
5180,1,26,61,2both,-0.209459,0
5181,1,26,62,2both,-0.736486,0
5182,1,26,63,2both,-0.736486,0


In [7]:
path_or_buf = data_behv + 'acc_change_lmm_subj_exp1.csv' # save in the folder of online data
acc_lmm_pd.to_csv(path_or_buf = path_or_buf)

## Fasle alarm (FA) rates (new Figure 2D and its corresponding SI in Experiment 1)

The data was originally saved in seqMemTask_v1_lureEffect.m

In [4]:
FAmat = loadmat(statSaveDir + 'FA_lure_overall_YAOA_exp1.mat') # FA_lure_overall_YAOA
FA_lmm_group = FAmat['FA_lure_overall_YAOA']
FA_lmm_pd    = pd.DataFrame(FA_lmm_group) 
FA_lmm_pd
# 5 columns
# (1) age groups: 0-YA, 1-OA
# (2) subj: 0-subLen
# (3) report types: 0-marginal reports, 1-reconstruction report
# (4) dimension: 0-item, 1-pos
# (5) False alarm (FA) rates due to the lure stimuli: continuous variable between 0 and 1


,0,1,2,3,4
0,0.0,0.0,0.0,0.0,0.078125
1,0.0,1.0,0.0,0.0,0.015625
2,0.0,2.0,0.0,0.0,0.015625
3,0.0,3.0,0.0,0.0,0.000000
4,0.0,4.0,0.0,0.0,0.015625
...,...,...,...,...,...
259,1.0,22.0,1.0,1.0,0.312500
260,1.0,23.0,1.0,1.0,0.546875
261,1.0,24.0,1.0,1.0,0.796875
262,1.0,25.0,1.0,1.0,0.484375


In [5]:
column_name = {0: 'groupNo', 1: 'subjNo', 2: 'reportType', 3: 'reportDim', 4: 'FArates'} 
FA_lmm_pd   = FA_lmm_pd.rename(columns = column_name) 
FA_lmm_pd.groupNo    = FA_lmm_pd.groupNo.astype('int64')
FA_lmm_pd.subjNo     = FA_lmm_pd.subjNo.astype('int64')
FA_lmm_pd.reportType = FA_lmm_pd.reportType.astype('int64')
FA_lmm_pd.reportDim  = FA_lmm_pd.reportDim.astype('int64')
FA_lmm_pd


,groupNo,subjNo,reportType,reportDim,FArates
0,0,0,0,0,0.078125
1,0,1,0,0,0.015625
2,0,2,0,0,0.015625
3,0,3,0,0,0.000000
4,0,4,0,0,0.015625
...,...,...,...,...,...
259,1,22,1,1,0.312500
260,1,23,1,1,0.546875
261,1,24,1,1,0.796875
262,1,25,1,1,0.484375


In [6]:
# 5 columns
# (1) age groups: 0-YA, 1-OA
# (2) subj: 0-subLen
# (3) report types: 0-marginal reports, 1-reconstruction report
# (4) dimension: 0-item, 1-pos
# (5) False alarm (FA) rates due to the lure stimuli: continuous variable between 0 and 1

nGroup = 2 # YA and OA
nReps  = 2 # marginal and reconstruction report
nBrec  = 2 # 2 transition dimension: item & pos
groupList  = ['0younger', '1older']
reportList = ['0marg', '1recons']
bindDirecList = ['0Item', '1Pos']
# column_name = {0: 'groupNo', 1: 'subjNo', 2: 'reportType', 3: 'reportDim', 4: 'FArates'} 
for iGrp in range(0, nGroup):
    FA_lmm_pd.loc[(FA_lmm_pd["groupNo"] == iGrp), "groupNo"] = groupList[iGrp] 
    
for iRep in range(0, nReps):
    FA_lmm_pd.loc[(FA_lmm_pd["reportType"] == iRep), "reportType"] = reportList[iRep]
    
for iBd in range(0, nBrec):
    FA_lmm_pd.loc[(FA_lmm_pd["reportDim"] == iBd), "reportDim"] = bindDirecList[iBd]

FA_lmm_pd


,groupNo,subjNo,reportType,reportDim,FArates
0,0younger,0,0marg,0Item,0.078125
1,0younger,1,0marg,0Item,0.015625
2,0younger,2,0marg,0Item,0.015625
3,0younger,3,0marg,0Item,0.000000
4,0younger,4,0marg,0Item,0.015625
...,...,...,...,...,...
259,1older,22,1recons,1Pos,0.312500
260,1older,23,1recons,1Pos,0.546875
261,1older,24,1recons,1Pos,0.796875
262,1older,25,1recons,1Pos,0.484375


In [7]:
path_or_buf = statSaveDir + 'FA_lure_overall_YAOA_exp1.csv' 
FA_lmm_pd.to_csv(path_or_buf = path_or_buf)

## Transition related measures (new Figure 2E and its corresponding SI in Experiment 1)

The data was originally saved in seqMemTask_v1_anal_summary.m

In [4]:
transAccMat = loadmat(statSaveDir + 'transAcc_overall_group_exp1.mat') # 'transAcc_overall_group'
transAcc_lmm_group = transAccMat['transAcc_overall_group']
transAcc_lmm_pd    = pd.DataFrame(transAcc_lmm_group) 
transAcc_lmm_pd
# 6 columns:
# (1) age groups: 0-YA, 1-OA
# (2) subj: 0-subLen
# (3) report types: 0-marginal reports, 1-reconstruction report
# (4) transition dimension: 0-item, 1-pos
# (5) transition conditions: 0-preceding response is incorrect, 1-preceding response is correct
# (6) transtion accuracy: a value ranging from 0 to 1

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,1.0,0.979839
1,0.0,1.0,0.0,0.0,1.0,0.996078
2,0.0,2.0,0.0,0.0,1.0,0.984127
3,0.0,3.0,0.0,0.0,1.0,0.983806
4,0.0,4.0,0.0,0.0,1.0,0.984000
...,...,...,...,...,...,...
523,1.0,22.0,1.0,1.0,0.0,0.317647
524,1.0,23.0,1.0,1.0,0.0,0.256250
525,1.0,24.0,1.0,1.0,0.0,0.246988
526,1.0,25.0,1.0,1.0,0.0,0.235294


In [5]:
column_name = {0: 'groupNo', 1: 'subjNo', 2: 'reportType', 3: 'transDim', 4: 'transConds', 5: 'transAcc'} 
transAcc_lmm_pd = transAcc_lmm_pd.rename(columns = column_name) 
transAcc_lmm_pd.groupNo    = transAcc_lmm_pd.groupNo.astype('int64')
transAcc_lmm_pd.subjNo     = transAcc_lmm_pd.subjNo.astype('int64')
transAcc_lmm_pd.reportType = transAcc_lmm_pd.reportType.astype('int64')
transAcc_lmm_pd.transDim   = transAcc_lmm_pd.transDim.astype('int64')
transAcc_lmm_pd.transConds = transAcc_lmm_pd.transConds.astype('int64')
transAcc_lmm_pd


,groupNo,subjNo,reportType,transDim,transConds,transAcc
0,0,0,0,0,1,0.979839
1,0,1,0,0,1,0.996078
2,0,2,0,0,1,0.984127
3,0,3,0,0,1,0.983806
4,0,4,0,0,1,0.984000
...,...,...,...,...,...,...
523,1,22,1,1,0,0.317647
524,1,23,1,1,0,0.256250
525,1,24,1,1,0,0.246988
526,1,25,1,1,0,0.235294


In [6]:
# 6 columns:
# (1) age groups: 0-YA, 1-OA
# (2) subj: 0-subLen
# (3) report types: 0-marginal reports, 1-reconstruction report
# (4) transition dimension: 0-item, 1-pos
# (5) transition conditions: 0-preceding response is incorrect, 1-preceding response is correct
# (6) transtion accuracy: a value ranging from 0 to 1

nGroup = 2 # YA and OA
nReps  = 2 # marginal and reconstruction report
nBrec  = 2 # 2 transition dimension: item & pos
nBcon  = 2 # 2 transition condition: preceding response incorrect (0) and preceding response correct (1)
groupList  = ['0younger', '1older']
reportList = ['0marg', '1recons']
bindDirecList = ['0Item', '1Pos']
bindCondsList = ['0PreWrong', '1PreCorrect']
# column_name = {0: 'groupNo', 1: 'subjNo', 2: 'reportType', 3: 'transDim', 4: 'transConds', 5: 'transAcc'} 
for iGrp in range(0, nGroup):
    transAcc_lmm_pd.loc[(transAcc_lmm_pd["groupNo"] == iGrp), "groupNo"] = groupList[iGrp] 
    
for iRep in range(0, nReps):
    transAcc_lmm_pd.loc[(transAcc_lmm_pd["reportType"] == iRep), "reportType"] = reportList[iRep]
    
for iBd in range(0, nBrec):
    transAcc_lmm_pd.loc[(transAcc_lmm_pd["transDim"] == iBd), "transDim"] = bindDirecList[iBd]
    
for iBc in range(0, nBcon):
    transAcc_lmm_pd.loc[(transAcc_lmm_pd["transConds"] == iBc), "transConds"] = bindCondsList[iBc]

transAcc_lmm_pd


,groupNo,subjNo,reportType,transDim,transConds,transAcc
0,0younger,0,0marg,0Item,1PreCorrect,0.979839
1,0younger,1,0marg,0Item,1PreCorrect,0.996078
2,0younger,2,0marg,0Item,1PreCorrect,0.984127
3,0younger,3,0marg,0Item,1PreCorrect,0.983806
4,0younger,4,0marg,0Item,1PreCorrect,0.984000
...,...,...,...,...,...,...
523,1older,22,1recons,1Pos,0PreWrong,0.317647
524,1older,23,1recons,1Pos,0PreWrong,0.256250
525,1older,24,1recons,1Pos,0PreWrong,0.246988
526,1older,25,1recons,1Pos,0PreWrong,0.235294


In [7]:
path_or_buf = statSaveDir + 'transAcc_overall_group_exp1.csv' 
transAcc_lmm_pd.to_csv(path_or_buf = path_or_buf)

## Binding related measures (Experiment 1)

### Figure 2H and 2I: overall binding score difference in Experiment 1

The data was originally saved in seqMemTask_v1_anal_summary.m 

In [4]:
bindsMat = loadmat(statSaveDir + 'binds_overall_diff_group_exp1.mat') # 'binds_overall_diff_group'
binds_lmm_group = bindsMat['binds_overall_diff_group']
binds_lmm_pd    = pd.DataFrame(binds_lmm_group) 
binds_lmm_pd
# 6 columns:
# (1) age groups: 0-YA, 1-OA
# (2) subj: 0-subLen
# (3) report types: 0-marginal reports, 1-reconstruction report
# (4) binding direction: 0-(item | pos), 1-(pos | item)
# (5) binding conditions: 0-binding cost (flipped, the same direction as benefit), 1-binding benefit
# (6) binding score: a value ranging from 0 to 1

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,0.0,0.237500
1,0.0,1.0,0.0,0.0,0.0,-0.003984
2,0.0,2.0,0.0,0.0,0.0,0.154472
3,0.0,3.0,0.0,0.0,0.0,-0.018868
4,0.0,4.0,0.0,0.0,0.0,-0.016461
...,...,...,...,...,...,...
523,1.0,22.0,1.0,1.0,1.0,0.268443
524,1.0,23.0,1.0,1.0,1.0,0.085714
525,1.0,24.0,1.0,1.0,1.0,0.223977
526,1.0,25.0,1.0,1.0,1.0,0.725869


In [5]:
column_name = {0: 'groupNo', 1: 'subjNo', 2: 'reportType', 3: 'bindDirec', 4: 'bindConds', 5: 'bindScore'} 
binds_lmm_pd = binds_lmm_pd.rename(columns = column_name) 
binds_lmm_pd.groupNo    = binds_lmm_pd.groupNo.astype('int64')
binds_lmm_pd.subjNo     = binds_lmm_pd.subjNo.astype('int64')
binds_lmm_pd.reportType = binds_lmm_pd.reportType.astype('int64')
binds_lmm_pd.bindDirec  = binds_lmm_pd.bindDirec.astype('int64')
binds_lmm_pd.bindConds  = binds_lmm_pd.bindConds.astype('int64')
binds_lmm_pd

,groupNo,subjNo,reportType,bindDirec,bindConds,bindScore
0,0,0,0,0,0,0.237500
1,0,1,0,0,0,-0.003984
2,0,2,0,0,0,0.154472
3,0,3,0,0,0,-0.018868
4,0,4,0,0,0,-0.016461
...,...,...,...,...,...,...
523,1,22,1,1,1,0.268443
524,1,23,1,1,1,0.085714
525,1,24,1,1,1,0.223977
526,1,25,1,1,1,0.725869


In [6]:
# 6 columns:
# (1) age groups: 0-YA, 1-OA
# (2) subj
# (3) report types: 0-marginal reports, 1-reconstruction report
# (4) binding direction: 0-(item | pos), 1-(pos | item)
# (5) binding conditions: 1-the other dimension is correct, 0-the other dimension is incorrect
# (6) binding score: a value ranging from 0 to 1

nGroup = 2 # YA and OA
nReps  = 2 # marginal and reconstruction report
nBrec  = 2 # 2 binding directions: (item | pos) & (pos | item)
nBcon  = 2 # 2 binding condition: cost (0, flipped, with the same direction as benefit) and benefit (1)
groupList  = ['0younger', '1older']
reportList = ['0marg', '1recons']
bindDirecList = ['0ItemOnPos', '1PosOnItem']
bindCondsList = ['0cost', '1benefit']
for iGrp in range(0, nGroup):
    binds_lmm_pd.loc[(binds_lmm_pd["groupNo"] == iGrp), "groupNo"] = groupList[iGrp] 
    
for iRep in range(0, nReps):
    binds_lmm_pd.loc[(binds_lmm_pd["reportType"] == iRep), "reportType"] = reportList[iRep]
    
for iBd in range(0, nBrec):
    binds_lmm_pd.loc[(binds_lmm_pd["bindDirec"] == iBd), "bindDirec"] = bindDirecList[iBd]
    
for iBc in range(0, nBcon):
    binds_lmm_pd.loc[(binds_lmm_pd["bindConds"] == iBc), "bindConds"] = bindCondsList[iBc]

binds_lmm_pd


,groupNo,subjNo,reportType,bindDirec,bindConds,bindScore
0,0younger,0,0marg,0ItemOnPos,0cost,0.237500
1,0younger,1,0marg,0ItemOnPos,0cost,-0.003984
2,0younger,2,0marg,0ItemOnPos,0cost,0.154472
3,0younger,3,0marg,0ItemOnPos,0cost,-0.018868
4,0younger,4,0marg,0ItemOnPos,0cost,-0.016461
...,...,...,...,...,...,...
523,1older,22,1recons,1PosOnItem,1benefit,0.268443
524,1older,23,1recons,1PosOnItem,1benefit,0.085714
525,1older,24,1recons,1PosOnItem,1benefit,0.223977
526,1older,25,1recons,1PosOnItem,1benefit,0.725869


In [7]:
path_or_buf = statSaveDir + 'binds_overall_diff_group_exp1.csv' 
binds_lmm_pd.to_csv(path_or_buf = path_or_buf)

### Figure 2G and the corresponding SI: overall binding score in Experiment 1

The data was originally saved in seqMemTask_v1_anal_summary.m 

In [8]:
bindsMat = loadmat(statSaveDir + 'binds_overall_group_exp1.mat') # 'binds_overall_group'
binds_lmm_group = bindsMat['binds_overall_group']
binds_lmm_pd    = pd.DataFrame(binds_lmm_group) 
binds_lmm_pd
# 6 columns:
# (1) age groups: 0-YA, 1-OA
# (2) subj: 0-subLen
# (3) report types: 0-marginal reports, 1-reconstruction report
# (4) binding direction: 0-(item | pos), 1-(pos | item)
# (5) binding conditions: 1-the other dimension is correct, 0-the other dimension is incorrect
# (6) binding score: a value ranging from 0 to 1

,0,1,2,3,4,5
0,0.0,0.0,0.0,0.0,1.0,0.987500
1,0.0,1.0,0.0,0.0,1.0,0.996016
2,0.0,2.0,0.0,0.0,1.0,0.987805
3,0.0,3.0,0.0,0.0,1.0,0.981132
4,0.0,4.0,0.0,0.0,1.0,0.983539
...,...,...,...,...,...,...
1051,1.0,22.0,1.0,1.0,0.0,0.125000
1052,1.0,23.0,1.0,1.0,0.0,0.200000
1053,1.0,24.0,1.0,1.0,0.0,0.144444
1054,1.0,25.0,1.0,1.0,0.0,0.036036


In [9]:
column_name = {0: 'groupNo', 1: 'subjNo', 2: 'reportType', 3: 'bindDirec', 4: 'bindConds', 5: 'bindScore'} 
binds_lmm_pd = binds_lmm_pd.rename(columns = column_name) 
binds_lmm_pd.groupNo    = binds_lmm_pd.groupNo.astype('int64')
binds_lmm_pd.subjNo     = binds_lmm_pd.subjNo.astype('int64')
binds_lmm_pd.reportType = binds_lmm_pd.reportType.astype('int64')
binds_lmm_pd.bindDirec  = binds_lmm_pd.bindDirec.astype('int64')
binds_lmm_pd.bindConds  = binds_lmm_pd.bindConds.astype('int64')
binds_lmm_pd

,groupNo,subjNo,reportType,bindDirec,bindConds,bindScore
0,0,0,0,0,1,0.987500
1,0,1,0,0,1,0.996016
2,0,2,0,0,1,0.987805
3,0,3,0,0,1,0.981132
4,0,4,0,0,1,0.983539
...,...,...,...,...,...,...
1051,1,22,1,1,0,0.125000
1052,1,23,1,1,0,0.200000
1053,1,24,1,1,0,0.144444
1054,1,25,1,1,0,0.036036


In [10]:
# 6 columns:
# (1) age groups: 0-YA, 1-OA
# (2) subj
# (3) report types: 0-marginal reports, 1-reconstruction report
# (4) binding direction: 0-(item | pos), 1-(pos | item)
# (5) binding conditions: 1-the other dimension is correct, 0-the other dimension is incorrect
# (6) binding score: a value ranging from 0 to 1

nGroup = 2 # YA and OA
nReps  = 2 # marginal and reconstruction report
nBrec  = 2 # 2 binding directions: (item | pos) & (pos | item)
nBcon  = 2 # 2 binding condition (their difference is 'cost' or 'benefit'): other dimension is Wrong (0) or Right (1)
groupList  = ['0younger', '1older']
reportList = ['0marg', '1recons']
bindDirecList = ['0ItemOnPos', '1PosOnItem']
bindCondsList = ['0OtherW', '1OtherR']
for iGrp in range(0, nGroup):
    binds_lmm_pd.loc[(binds_lmm_pd["groupNo"] == iGrp), "groupNo"] = groupList[iGrp] 
    
for iRep in range(0, nReps):
    binds_lmm_pd.loc[(binds_lmm_pd["reportType"] == iRep), "reportType"] = reportList[iRep]
    
for iBd in range(0, nBrec):
    binds_lmm_pd.loc[(binds_lmm_pd["bindDirec"] == iBd), "bindDirec"] = bindDirecList[iBd]
    
for iBc in range(0, nBcon):
    binds_lmm_pd.loc[(binds_lmm_pd["bindConds"] == iBc), "bindConds"] = bindCondsList[iBc]

binds_lmm_pd

,groupNo,subjNo,reportType,bindDirec,bindConds,bindScore
0,0younger,0,0marg,0ItemOnPos,1OtherR,0.987500
1,0younger,1,0marg,0ItemOnPos,1OtherR,0.996016
2,0younger,2,0marg,0ItemOnPos,1OtherR,0.987805
3,0younger,3,0marg,0ItemOnPos,1OtherR,0.981132
4,0younger,4,0marg,0ItemOnPos,1OtherR,0.983539
...,...,...,...,...,...,...
1051,1older,22,1recons,1PosOnItem,0OtherW,0.125000
1052,1older,23,1recons,1PosOnItem,0OtherW,0.200000
1053,1older,24,1recons,1PosOnItem,0OtherW,0.144444
1054,1older,25,1recons,1PosOnItem,0OtherW,0.036036


In [11]:
path_or_buf = statSaveDir + 'binds_overall_group_exp1.csv' 
binds_lmm_pd.to_csv(path_or_buf = path_or_buf)

# Experiment 2: only Interleaved curriculum is relevant -- three trial organization conditions (interleaved, contentBlocked, positionBlocked)

<div class="alert alert-success">
<b>(Columns) variables</b>:
    
{0: 'groupNo', 1: 'subjNo', 2: 'trialNo', 3: 'measNo', 4: 'accuracy', 5: 'fullyCor', 6: 'order'}

groupNo: younger or older
    
measNo: content, position, and reconstruction
    
order: chronological order after the encoding stage, could be 1, 2, & 3
</div>

## Decline change percentage (older minus younger) across three learning curriculums

The data was originally saved in seqMemTask_v2_anal_summary.m

In [4]:
groupName = 'older'
acc_change_mat = loadmat(data_behv + 'acc_change_conds_group_' + groupName + '_exp2.mat') # 'acc_change_conds_group'
acc_lmm_group = acc_change_mat['acc_change_conds_group']
acc_lmm_pd    = pd.DataFrame(acc_lmm_group) 
acc_lmm_pd


,0,1,2,3,4,5,6
0,0.0,0.0,1.0,0.0,0.122588,0.0,2.0
1,0.0,0.0,2.0,0.0,0.122588,0.0,1.0
2,0.0,0.0,3.0,0.0,0.122588,0.0,2.0
3,0.0,0.0,4.0,0.0,0.122588,0.0,1.0
4,0.0,0.0,6.0,0.0,0.122588,0.0,1.0
...,...,...,...,...,...,...,...
11675,2.0,24.0,60.0,2.0,0.260551,0.0,3.0
11676,2.0,24.0,61.0,2.0,0.260551,0.0,1.0
11677,2.0,24.0,62.0,2.0,0.260551,0.0,3.0
11678,2.0,24.0,63.0,2.0,0.260551,0.0,3.0


In [5]:
column_name = {0: 'groupNo', 1: 'subjNo', 2: 'trialNo', 3: 'measNo', 4: 'accuracy_change', 5: 'fullyCor', 6: 'order'} 
acc_lmm_pd = acc_lmm_pd.rename(columns = column_name) 
acc_lmm_pd.groupNo = acc_lmm_pd.groupNo.astype('int64')
acc_lmm_pd.subjNo  = acc_lmm_pd.subjNo.astype('int64')
acc_lmm_pd.trialNo = acc_lmm_pd.trialNo.astype('int64')
acc_lmm_pd.measNo  = acc_lmm_pd.measNo.astype('int64')
acc_lmm_pd.fullyCor = acc_lmm_pd.fullyCor.astype('int64')
acc_lmm_pd.order = acc_lmm_pd.order.astype('int64')
acc_lmm_pd


,groupNo,subjNo,trialNo,measNo,accuracy_change,fullyCor,order
0,0,0,1,0,0.122588,0,2
1,0,0,2,0,0.122588,0,1
2,0,0,3,0,0.122588,0,2
3,0,0,4,0,0.122588,0,1
4,0,0,6,0,0.122588,0,1
...,...,...,...,...,...,...,...
11675,2,24,60,2,0.260551,0,3
11676,2,24,61,2,0.260551,0,1
11677,2,24,62,2,0.260551,0,3
11678,2,24,63,2,0.260551,0,3


In [6]:
nGroup = 3
nMeas  = 3
nOrder = 3
groupList = ['0interleaved', '1contentBlocked', '2positionBlocked']
measList  = ['0content', '1position', '2both']
orderList = ['0first', '1second', '2third']
for iM in range(0, nMeas):
    acc_lmm_pd.loc[(acc_lmm_pd["measNo"] == iM), "measNo"] = measList[iM]
for iGrp in range(0, nGroup):
    acc_lmm_pd.loc[(acc_lmm_pd["groupNo"] == iGrp), "groupNo"] = groupList[iGrp]  
for iOrd in range(0, nOrder):
    acc_lmm_pd.loc[(acc_lmm_pd["order"] == (iOrd + 1)), "order"] = orderList[iOrd]

In [7]:
acc_lmm_pd

,groupNo,subjNo,trialNo,measNo,accuracy_change,fullyCor,order
0,0interleaved,0,1,0content,0.122588,0,1second
1,0interleaved,0,2,0content,0.122588,0,0first
2,0interleaved,0,3,0content,0.122588,0,1second
3,0interleaved,0,4,0content,0.122588,0,0first
4,0interleaved,0,6,0content,0.122588,0,0first
...,...,...,...,...,...,...,...
11675,2positionBlocked,24,60,2both,0.260551,0,2third
11676,2positionBlocked,24,61,2both,0.260551,0,0first
11677,2positionBlocked,24,62,2both,0.260551,0,2third
11678,2positionBlocked,24,63,2both,0.260551,0,2third


In [8]:
path_or_buf = data_behv + 'acc_change_conds_group_' + groupName + '_exp2.csv' # save in the folder of online data
acc_lmm_pd.to_csv(path_or_buf = path_or_buf)

## Figure 3C relevant: choice in each slot as dependent variables (GLMM)

### Figure 3C: Choice (correct-1, incorrect-0) in each slot; concatenate YA and OA

The data was originally saved in seqMemTask_v2_anal_summary.m

#### Relabel each column for YA and OA separately

In [9]:
groupId = 2
if groupId == 1:  # younger participants
    groupName = 'younger'
elif groupId == 2: # older participants
    groupName = 'older'
    

In [10]:
choiceMat = loadmat(data_behv + 'choice_lmm_group_conds_' + groupName + '_exp2.mat') # 'choice_lmm_group'
choice_lmm_group = choiceMat['choice_lmm_group']
choice_lmm_pd    = pd.DataFrame(choice_lmm_group) 
choice_lmm_pd


,0,1,2,3,4,5,6
0,0,0,1,0,0,1,1
1,0,0,1,0,1,1,1
2,0,0,1,0,2,1,1
3,0,0,1,0,3,1,1
4,0,0,1,0,4,1,1
...,...,...,...,...,...,...,...
58395,2,24,64,2,0,1,2
58396,2,24,64,2,1,1,2
58397,2,24,64,2,2,1,2
58398,2,24,64,2,3,1,2


In [11]:
column_name = {0: 'groupNo', 1: 'subjNo', 2: 'trialNo', 3: 'measNo', 4: 'slotNo', 5: 'choice', 6: 'order'} 
choice_lmm_pd = choice_lmm_pd.rename(columns = column_name) 
choice_lmm_pd.groupNo = choice_lmm_pd.groupNo.astype('int64')
choice_lmm_pd.subjNo  = choice_lmm_pd.subjNo.astype('int64')
choice_lmm_pd.trialNo = choice_lmm_pd.trialNo.astype('int64')
choice_lmm_pd.measNo  = choice_lmm_pd.measNo.astype('int64')
choice_lmm_pd.slotNo  = choice_lmm_pd.slotNo.astype('int64')
choice_lmm_pd.choice  = choice_lmm_pd.choice.astype('int64')
choice_lmm_pd.order   = choice_lmm_pd.order.astype('int64')
choice_lmm_pd


,groupNo,subjNo,trialNo,measNo,slotNo,choice,order
0,0,0,1,0,0,1,1
1,0,0,1,0,1,1,1
2,0,0,1,0,2,1,1
3,0,0,1,0,3,1,1
4,0,0,1,0,4,1,1
...,...,...,...,...,...,...,...
58395,2,24,64,2,0,1,2
58396,2,24,64,2,1,1,2
58397,2,24,64,2,2,1,2
58398,2,24,64,2,3,1,2


In [12]:
nGroup = 3
nMeas  = 3
nSlot  = 5
nOrder = 3
groupList = ['0interleaved', '1contentBlocked', '2positionBlocked']
measList  = ['0content', '1position', '2both']
slotList  = ['0first', '1second', '2third', '3fourth', '4fifth']
orderList = ['0firstWin', '1secondWin', '2thirdWin']

for iM in range(0, nMeas):
    choice_lmm_pd.loc[(choice_lmm_pd["measNo"] == iM), "measNo"] = measList[iM]
for iGrp in range(0, nGroup):
    choice_lmm_pd.loc[(choice_lmm_pd["groupNo"] == iGrp), "groupNo"] = groupList[iGrp]  
for iSlt in range(0, nSlot):
    choice_lmm_pd.loc[(choice_lmm_pd["slotNo"] == iSlt), "slotNo"] = slotList[iSlt]
for iOrd in range(0, nOrder):
    choice_lmm_pd.loc[(choice_lmm_pd["order"] == iOrd), "order"] = orderList[iOrd]
    

In [13]:
path_or_buf = data_behv + 'choice_lmm_group_conds_' + groupName + '_exp2.csv' # save in the folder of online data
choice_lmm_pd.to_csv(path_or_buf = path_or_buf)

#### Concatenate the data from YA and OA

In [15]:
path_or_buf = data_behv + 'choice_lmm_group_conds_' + 'younger' + '_exp2.csv'
choice_lmm_pd_YA = pd.read_csv(path_or_buf, index_col=0)
choice_lmm_pd_YA['trialNo']  = np.log(choice_lmm_pd_YA['trialNo'])
choice_lmm_pd_YA['groupAge'] = 'younger'
choice_lmm_pd_YA

,groupNo,subjNo,trialNo,measNo,slotNo,choice,order,groupAge
0,0interleaved,0,0.000000,0content,0first,1,1secondWin,younger
1,0interleaved,0,0.000000,0content,1second,1,1secondWin,younger
2,0interleaved,0,0.000000,0content,2third,1,1secondWin,younger
3,0interleaved,0,0.000000,0content,3fourth,1,1secondWin,younger
4,0interleaved,0,0.000000,0content,4fifth,1,1secondWin,younger
...,...,...,...,...,...,...,...,...
55195,2positionBlocked,20,4.158883,2both,0first,1,2thirdWin,younger
55196,2positionBlocked,20,4.158883,2both,1second,1,2thirdWin,younger
55197,2positionBlocked,20,4.158883,2both,2third,1,2thirdWin,younger
55198,2positionBlocked,20,4.158883,2both,3fourth,1,2thirdWin,younger


In [16]:
path_or_buf = data_behv + 'choice_lmm_group_conds_' + 'older' + '_exp2.csv'
choice_lmm_pd_OA = pd.read_csv(path_or_buf, index_col=0)
choice_lmm_pd_OA['trialNo']  = np.log(choice_lmm_pd_OA['trialNo'])
choice_lmm_pd_OA['groupAge'] = 'older' 
choice_lmm_pd_OA

,groupNo,subjNo,trialNo,measNo,slotNo,choice,order,groupAge
0,0interleaved,0,0.000000,0content,0first,1,1secondWin,older
1,0interleaved,0,0.000000,0content,1second,1,1secondWin,older
2,0interleaved,0,0.000000,0content,2third,1,1secondWin,older
3,0interleaved,0,0.000000,0content,3fourth,1,1secondWin,older
4,0interleaved,0,0.000000,0content,4fifth,1,1secondWin,older
...,...,...,...,...,...,...,...,...
58395,2positionBlocked,24,4.158883,2both,0first,1,2thirdWin,older
58396,2positionBlocked,24,4.158883,2both,1second,1,2thirdWin,older
58397,2positionBlocked,24,4.158883,2both,2third,1,2thirdWin,older
58398,2positionBlocked,24,4.158883,2both,3fourth,1,2thirdWin,older


In [17]:
# Concatenate both datasets
choice_lmm_pd = pd.concat([choice_lmm_pd_YA, choice_lmm_pd_OA], ignore_index=True)
choice_lmm_pd

,groupNo,subjNo,trialNo,measNo,slotNo,choice,order,groupAge
0,0interleaved,0,0.000000,0content,0first,1,1secondWin,younger
1,0interleaved,0,0.000000,0content,1second,1,1secondWin,younger
2,0interleaved,0,0.000000,0content,2third,1,1secondWin,younger
3,0interleaved,0,0.000000,0content,3fourth,1,1secondWin,younger
4,0interleaved,0,0.000000,0content,4fifth,1,1secondWin,younger
...,...,...,...,...,...,...,...,...
113595,2positionBlocked,24,4.158883,2both,0first,1,2thirdWin,older
113596,2positionBlocked,24,4.158883,2both,1second,1,2thirdWin,older
113597,2positionBlocked,24,4.158883,2both,2third,1,2thirdWin,older
113598,2positionBlocked,24,4.158883,2both,3fourth,1,2thirdWin,older


In [18]:
path_or_buf = data_behv + 'choice_lmm_group_conds_YA_OA_exp2.csv' # save in the folder of online data
choice_lmm_pd.to_csv(path_or_buf = path_or_buf)

## Binding related measures (Experiment 2)